# FINM 32000 Numerical Methods HW3

---- Yitong Wang

In [ ]:
import numpy as np
from scipy.sparse import diags
from scipy.sparse.linalg import spsolve

## Problem 1

#### (a)

- CEV dynamics:
$$ dS_t = \sigma S_t^{1+\alpha}dW_t$$
- GBM dynamics:
$$ dS_t = r S_t dt + \sigma S_t dW_t$$
- Thus, they can be writtern as dynamics:
$$ \textcolor{orange}{dS_t = R_{grow} S_t dt + \sigma S_t^{1+\alpha} dW_t}$$
- where
    - in CEV, $R_{grow}=0;$ 
    - in GBM, $R_{grow}=r; \alpha=0$


- Apply Ito's furmula on $C(S_t, t)$: 
$$ d C(S_t, t) = \frac{\partial C}{\partial t} dt + \frac{\partial C}{\partial S} dS_t + \frac{1}{2} \frac{\partial^2 C}{\partial S^2} dS_t^2$$
$$ \Rightarrow d C(S_t, t) =  [\frac{\partial C}{\partial t} + \frac{\partial C}{\partial S} R_{grow} S_t + \frac{1}{2} \frac{\partial^2 C}{\partial S^2} \sigma^2 S_t^{2(1+\alpha)}] dt + \frac{\partial C}{\partial S} \sigma S_t^{1+\alpha} dW_t $$
- Set the drift term to be $rC(S_t, t)$ and derive the PDE:
$$ \frac{\partial C}{\partial t} + \frac{\partial C}{\partial S} R_{grow} S_t + \frac{1}{2} \frac{\partial^2 C}{\partial S^2} \sigma^2 S_t^{2(1+\alpha)} = rC $$
- Since the contract is a put on $S$:
$$
\begin{cases}
\frac{\partial C}{\partial t} + \frac{\partial C}{\partial S} R_{grow} S_t + \frac{1}{2} \frac{\partial^2 C}{\partial S^2} \sigma^2 S_t^{2(1+\alpha)} = rC \quad \text{(PDE)}\\
C(S_T, T) = (K-S_T)^+ \quad \text{(terminal condition)}
\end{cases}
$$

In [7]:
class CEV:

    def __init__(self,volcoeff,alpha,rGrow,r,S0):
        self.volcoeff = volcoeff
        self.alpha = alpha
        self.rGrow = rGrow # for futures price, rGrow=0
        self.r = r
        self.S0 = S0

In [8]:
class Put:

    def __init__(self,T,K):
        self.T = T
        self.K = K

### Crank-Nicolson
- We want to solve for $C(x,t)$ the full PDE (not just model problem)
$$ \frac{\partial C}{\partial t} + f(x,t)\frac{\partial^2 C}{\partial x^2} + g(x,t)\frac{\partial C}{\partial x} + h(x,t)C = 0 $$

- With given $f,g,h$, and given terminal conditions. Let
$$ f_{n}^{j} := f(x_j, t_n), \quad g_{n}^{j} := g(x_j, t_n), \quad h_{n}^{j} := h(x_j, t_n) $$
- Then for $j= -(J-1), ..., J-1$,
$$
-F_{n}^{j} C_{n}^{j+1} + (1 + G_{n}^{j}) C_{n}^{j} - H_{n}^{j} C_{n}^{j-1} = F_{n+1}^{j} C_{n+1}^{j+1} + (1 - G_{n+1}^{j}) C_{n+1}^{j} + H_{n+1}^{j} C_{n+1}^{j-1}
$$
where
$$
\begin{align*}
F_{n}^{j} &:= \frac{1}{2} \frac{\Delta t}{(\Delta x)^2} f_{n}^{j} + \frac{1}{4} \frac{\Delta t}{\Delta x} g_{n}^{j} \\
G_{n}^{j} &:= \frac{\Delta t}{(\Delta x)^2} f_{n}^{j} - \frac{\Delta t}{2}  h_{n}^{j} \\
H_{n}^{j} &:= \frac{1}{2} \frac{\Delta t}{(\Delta x)^2} f_{n}^{j} - \frac{1}{4} \frac{\Delta t}{\Delta x} g_{n}^{j} \\
\end{align*}
$$
- Assume that the boundary values $C_{n}^{J}$ and $C_{n}^{-J}$ for $n = 0,...,N$ are given. 
    - For example, for a put option with strike $K$ (if using price $S$ it self in PDE rather than $x=lnS$):
$$ 
\begin{cases}
C_n^{J} = 0 \\
C_n^{-J} = Ke^{-r(T-t_n)} - S_{-J} 
\end{cases}$$

- in matrix form,
$$
\begin{pmatrix}
1 + G_{n}^{J-1} & -H_{n}^{J-1} &  &  &  \\
-F_{n}^{J-2} & 1 + G_{n}^{J-2} & -H_{n}^{J-2} &  &  \\
 & -F_{n}^{J-3} & 1 + G_{n}^{J-3} & \ddots &  \\
 &  & \ddots & \ddots & -H_{n}^{-J+2} \\
 &  &  & -F_{n}^{-J+1} & 1 + G_{n}^{-J+1}
\end{pmatrix}
\textcolor{cyan}{
\begin{pmatrix}
C_{n}^{J-1} \\
C_{n}^{J-2} \\
\vdots \\
C_{n}^{-J+1}
\end{pmatrix}}
+
\begin{pmatrix}
-F_{n}^{J-1} C_{n}^{J} \\
0 \\
\vdots \\
0 \\
-H_{n}^{-J+1} C_{n}^{-J}
\end{pmatrix}
$$

$$
=
\begin{pmatrix}
1 - G_{n+1}^{J-1} & H_{n+1}^{J-1} &  &  &  \\
F_{n+1}^{J-2} & 1 - G_{n+1}^{J-2} & H_{n+1}^{J-2} &  &  \\
 & F_{n+1}^{J-3} & 1 - G_{n+1}^{J-3} & \ddots &  \\
 &  & \ddots & \ddots & H_{n+1}^{-J+2} \\
 &  &  & F_{n+1}^{-J+1} & 1 - G_{n+1}^{-J+1}
\end{pmatrix}
\textcolor{red}{
\begin{pmatrix}
C_{n+1}^{J-1} \\
C_{n+1}^{J-2} \\
\vdots \\
C_{n+1}^{-J+1}
\end{pmatrix}}
+
\begin{pmatrix}
F_{n+1}^{J-1} C_{n+1}^{J} \\
0 \\
\vdots \\
0 \\
H_{n+1}^{-J+1} C_{n+1}^{-J}
\end{pmatrix}
$$

If given terminal conditions, then we know  $\textcolor{red}{C_{n+1}}$ and we solve for $\textcolor{cyan}{C_n}$.



#### (b)
- Thus, for CEV (including GBM) dynamics:
$$ \frac{\partial C}{\partial t} + \frac{1}{2} \sigma^2 S_t^{2(1+\alpha)} \frac{\partial^2 C}{\partial S^2} + R_{grow} S_t \frac{\partial C}{\partial S} - rC = 0 $$
$$ 
\begin{cases}
f_{n}^{j} = f(S_j, t_n) = \frac{1}{2} \sigma^2 {S_n^j}^{2(1+\alpha)} \\
g_{n}^{j} = g(x_j, t_n) =  R_{grow} S_t \\
h_{n}^{j} = h(x_j, t_n) = -r
\end{cases}
$$


In [102]:
class FD_CrankNicolson_Engine:

    def __init__(self,SMax,SMin,deltaS,deltat):
        self.SMax=SMax
        self.SMin=SMin
        self.deltaS=deltaS
        self.deltat=deltat

    def price_put_CEV(self,contract,dynamics):
    # returns array of all initial spots,
    # and the corresponding array of put prices

        alpha, r, rGrow, volcoeff = dynamics.alpha, dynamics.r, dynamics.rGrow, dynamics.volcoeff

        # SMin and SMax denote the smallest and largest S in the _interior_.
        # The boundary conditions are imposed one level _beyond_,
        # e.g. at S_lowboundary=SMin-deltaS, not at SMin.
        # To relate to lecture notation, S_lowboundary is S_{-J}
        # whereas SMin is S_{-J+1}

        N = round(contract.T/self.deltat) # num of time steps
        if abs(N-contract.T/self.deltat) > 1e-12:
            raise ValueError('Bad time step')
        
        numS = round((self.SMax-self.SMin)/self.deltaS) + 1 # num of price steps without the 2 boundary levels (2J-1), also the number of unknowns
        if abs(numS-(self.SMax-self.SMin)/self.deltaS-1) > 1e-12:
            raise ValueError('Bad price step')
        
        # from SMax to SMin (include SMin defaultly), generate numS size of numeber uniformly
        S = np.linspace(self.SMax, self.SMin, numS)    #The FIRST indices in this array are for HIGH levels of S
        
        S_lowboundary = self.SMin - self.deltaS # S_{-J}

        putprice=np.maximum(contract.K-S,0)

        ratio1 = self.deltat/self.deltaS
        ratio2 = self.deltat/self.deltaS**2
        f = (1/2) * (volcoeff**2) * (S ** (2*(1+alpha))) # You fill in with an array of the same size as S.
        g = rGrow * S  
        h = -r
        F = 0.5*ratio2*f + 0.25*ratio1*g
        G =     ratio2*f - 0.50*self.deltat*h
        H = 0.5*ratio2*f - 0.25*ratio1*g

        # Right-hand-side matrix
        # diags is a function to create SPARS matrix
        # H[:-1] because H_{J-1} to H_{-J+2} (the whole should be H_{J-1} to H_{-J+1})
        # F[1:] because F_{J-2} to H_{-J+1} (the whole should be F_{J-1} to F_{-J+1})
        # [1,0,-1] denotes for the diagonal line, the first line above the diagnoal line and the first line below the diagnoal line
        # numS = 2J-1, which is the shape of the matrix
        RHSmatrix = diags([H[:-1], 1-G, F[1:]], [1,0,-1], shape=(numS,numS), format="csr")

        #Left-hand-side matrix
        LHSmatrix = diags([-H[:-1], 1+G, -F[1:]], [1,0,-1], shape=(numS,numS), format="csr")

        # t step from N-1 to 0
        for t in np.arange(N-1,-1,-1)*self.deltat:

            rhs = RHSmatrix * putprice

            # Now let's add the boundary condition vectors.
            # They are nonzero only in the last component (because C_{n,J}=0, C_{n+1,J}=0):
            # Thus only need to update the last component
            # multiply 2 because in lhs, we also need to add the boundary condition vectors, but we directly minus it to the rhs
            
            rhs[-1] = rhs[-1] + 2*H[-1]*(contract.K-S_lowboundary)

            putprice = spsolve(LHSmatrix, rhs)
            # numpy.linalg.solve, which expects arrays as inputs,
            # is fine for small matrix equations, and for matrix equations without special structure.
            # But for large matrix equations in which the matrix has special structure,
            # we may want a more intelligent solver that can run faster
            # by taking advantage of the special structure of the matrix.
            # Specifically, in this case, let's try to use a solver that recognizes the SPARSE MATRIX structure.
            # Try spsolve, imported from scipy.sparse.linalg

            putprice = np.maximum(putprice, contract.K-S)

        return(S, putprice)
    
    def calc_time0_greeks(self,contract,dynamics):
        (S0_all, putprice0_all) = self.price_put_CEV(contract,dynamics)

        S_start = dynamics.S0-self.deltaS*1.5 # use 1.5 as a multiplier to give some redundant place
        S_end = dynamics.S0+self.deltaS*1.5
        between_rows = (S0_all>S_start) & (S0_all<S_end)
        calc_greeks = np.stack((S0_all, putprice0_all),axis=1)[between_rows]
        dS = calc_greeks[0, 0] - calc_greeks[1, 0]
        delta_numerator = calc_greeks[0, 1] - calc_greeks[2, 1]
        gamma_numerator = calc_greeks[0, 1] + calc_greeks[2, 1] - 2*calc_greeks[1, 1]
        delta = delta_numerator/(2*dS)
        gamma = gamma_numerator/(dS**2)
        print(f'time-0 delta is {np.round(delta,4)}, time-0 gamma is {np.round(gamma,4)}')
        

In [103]:
hw4dynamics = CEV(volcoeff=3, alpha=-0.5, rGrow=0, r=0.05, S0=100)

hw4contract=Put(T=0.25,K=100)

hw4FD = FD_CrankNicolson_Engine(SMax=200,SMin=50,deltaS=0.1,deltat=0.0005)

In [104]:
(S0_all, putprice) = hw4FD.price_put_CEV(hw4contract,hw4dynamics)

#### (c)
- The time-0 price is 5.9183
- The time-0 delta is -0.4806, and the time-0 gamma is 0.0264

In [107]:
# pricer_put_CEV_CrankNicolson gives us option prices for ALL S0 from SMin to SMax
# But let's display only for a few S0 near 100:

displayStart = hw4dynamics.S0-hw4FD.deltaS*1.5
displayEnd   = hw4dynamics.S0+hw4FD.deltaS*1.5
displayrows  = (S0_all>displayStart) & (S0_all<displayEnd)
np.set_printoptions(precision=4, suppress=True)
print(np.stack((S0_all, putprice),axis=1)[displayrows])

[[100.1      5.8704]
 [100.       5.9183]
 [ 99.9      5.9665]]


In [108]:
hw4FD.calc_time0_greeks(hw4contract,hw4dynamics)

time-0 delta is -0.4806, time-0 gamma is 0.0264


#### (d)

- The time-0 price is 5.442.
- The time-0 delta is -0.4486, and the time-0 gamma is 0.0275

In [113]:
gbmdynamics = CEV(volcoeff=0.3, alpha=0, rGrow=0.05, r=0.05, S0=100)

hw4contract=Put(T=0.25,K=100)

gbmFD = FD_CrankNicolson_Engine(SMax=200,SMin=50,deltaS=0.1,deltat=0.0005)


In [114]:
(S0_all, gbm_putprice) = gbmFD.price_put_CEV(hw4contract,gbmdynamics)

In [115]:
displayStart = hw4dynamics.S0-gbmFD.deltaS*1.5
displayEnd   = hw4dynamics.S0+gbmFD.deltaS*1.5
displayrows  = (S0_all>displayStart) & (S0_all<displayEnd)
np.set_printoptions(precision=4, suppress=True)
print(np.stack((S0_all, gbm_putprice),axis=1)[displayrows])

[[100.1      5.3973]
 [100.       5.442 ]
 [ 99.9      5.487 ]]


In [116]:
gbmFD.calc_time0_greeks(hw4contract,gbmdynamics)

time-0 delta is -0.4486, time-0 gamma is 0.0275


## Problem 2

![Problem2-1](https://github.com/Sally-Yitong/FINM-32000-Numerical-Methods/blob/main/HW4/1.jpg?raw=true)

![Problem2-2](https://github.com/Sally-Yitong/FINM-32000-Numerical-Methods/blob/main/HW4/2.jpg?raw=true)

- the standard deviation is smaller than the standard deviation in part (b), and it makes sense, since every draw in (c) should be negatively correlated with each other.